# Reinforcement learning & contextual bandits
- Petr Stanislav & Michal Kubišta
- [dataclair.ai](https://dataclair.ai/)

<img src="www/dcl_logo.jpg" align="middle" width="600" height="600">

## Structure
- Intro to reinforcement learning
- Multi-armed bandits
- Contextual bandits
- TF-Agents

# Intro to Reinforcement Learning (RL)

## Why do we care?
**assumption:** we have mastered all aspects of supervised learning for any kind of data type (text, images, tabular, ...)


**problem:** build a purely ML system to play perfect chess (using supervised learning)  
**input:** picture of chessboard  
**output:** which move to make  

This should be easy, right?

## Problems?

1. **what's the best move?**
<img src="www/best_move.png" align="middle" width="300" height="300" />

## Problems?
1. **what's the best move?**
1. **time(step)-dependency**
<img src="www/trebuchet.png" align="middle" width="300" height="300" />

## Problems?
1. **what's the best move?**
1. **time(step)-dependency**
1. **training data**
<img src="www/chessboard.png" align="middle" width="300" height="300" />

## Solutions?
1. **what's the best move?**
    - what if we instead assign specific rewards for different outcomes?
    - chess example:
        - if we take a piece &#8594; value of that piece (1, 3, 5, 9)
        - if we win &#8594; 30
        - otherwise &#8594; 0
1. time(step)-dependency
1. training data

## Why reward? (problem 1)
- we don't know the "true" result
- but we can define a general process to evaluate the actions we take
    - chess - taking a piece, winning the game
    - autonomous car - moving in the right direction, not causing any crashes
- we want to maximise the sum of all received rewards (**total reward**)
    - $TR_T = \sum_{t=1}^{T}r_t$
- `value` vs `reward`
    - when taking the action $a$ in state $s$:
        - `reward` is the observed immediate feedback
        - `value` is predicted long-term potential
            - an approximation of the **total reward**

## Is reward enough? (problem 1)

![](www/reward_is_enough.png)

## Limitation of reward (problem 1)
- credit assignment
    - how to match the future rewards to the actions that lead to them
    - e.g. we win a chess game (large positive reward), now we need to credit the actions that has helped us the most
    - let's review two chess positions:
        - [scholar's mate](https://lichess.org/analysis)
        - [back rank mate](https://lichess.org/analysis/k1rq2nr/pp4pp/3p4/8/8/3P4/PPQ3PP/K1R3NR_w_Kk_-_0_1)

- [perverse incentive](https://en.wikipedia.org/wiki/Perverse_incentive) (cobra effect)
    - [The Surprising Creativity of Digital Evolution](https://arxiv.org/abs/1803.03453)

## [HalfCheetah](https://www.alexirpan.com/2018/02/14/rl-hard.html)  (problem 1)


<video width="960" height="480" controls>
<source src="https://www.alexirpan.com/public/rl-hard/upsidedown_half_cheetah.mp4" type="video/mp4">
</video>

## Solutions?
1. what's the best move?
1. time(step)-dependency
1. **training data**
    - what if we let the algorithm generate it's own data?


## Generating the training dataset (problem 3)
- what we need
    - some **model** capable of online learning
        - get **input**, provide **decission**
    - some **engine** that will:
        - provide **input** (observation)
        - calculate the **reward** based on the model **decission** (prediction)
- we collect the data
    - have the engine generate the **inputs**
    - get the model to predict (randomly) the **decission**
    - collect the **reward** for each **decission**
    - save all of it - `(input, decission, reward)`
- after we collect enough data we use it to train the **model** and go collect more

## With RL terms (problem 3)
- we need some simulation - **environment**
    - includes definition of possible `actions`
    - chess engine with basic rules, or physical engine for robots
- the **enviroment** produces some observation - `state`
    - e.g. position of all pieces
- we need model that can decide based on the `observation` - **agent**
    - oracle - regression model that predicts the `value` of each `action` based on `observation`
    - policy - decide on the `action` based on predictions from oracle
- the agent plays the action and receives `reward` and a new `state` from the **environment**


<img src="www/RL_feedback_loop.jpg" align="middle" width="400" height="400">

## Why separate oracle and policy? (problem 3)
- we start with sequential classification
- for some reason turn it into a regression
- and attach some decission process on top of it?

### Exploration vs Exploitation
- we have (some degree of) control over how our dataset will look like
- this let's us decide between:
    - exploration - let the policy select sub-optimal action to see what happens
    - exploitation - make the policy select the action with highest predicted reward
- the task of the policy is to try to find the optimal balance between exploration & exploitation

## Solutions?
1. what's the best move?
1. **time(step)-dependency**
    - ensure the model can handle long time-series
1. training data

## Time dependency (poblem 2)
- the **reward** and future **state** depends on the current **state** and the **action** taken!
    - $(s_{t+1}, r_{t}) = f(a_t, s_t)$
    - e.g. for chess
        - $s_t$ - state of the chessboard now (position of all pieces)
        - $a_t$ - the move we make
        - $s_{t+1}$ - chessboard in the next time step
- our task is maximise the **total reward**
    - $TR_T = \sum_{t=1}^{t=T}r_t$
    - **agent** learns to estimate the `value` which is an approximation of $TR$
    - we can estimate both the value of state (`value` function) and action-state (`action value` function)

## What is RL?
- a study of how "intelligent" **agents** should take **actions** within an **environment** to collect maximum cummulative **reward**
- sequential classification problem, where each decission (prediction) results in some reward
    - we are trying to maximise the sum of these rewards
- often defined as Markov Decission Process (MDP)
    - $S$, $A$, $P_a(s'|s)$, $R_a(s'|s)$
    - in this case, solutions are often based on concepts of dynammic programming (Bellman equation)

# Contextual Multi-Armed Bandits (CMAB)

<img src="https://diydilettante.files.wordpress.com/2011/07/the-office-pictures.jpg">

###### credits: Murder (The Office)

# Contextual Multi-Armed Bandits (CMAB)
<img src="https://slivkins.com/work/bandits-svc/MAB-2.jpg">

###### credits: Alex Slivkins - Microsoft Research Silicon Valley

## CMAB vs RL?
- simplification of the RL problem
    - remove the long-term dependency
    - the future **state** no longer depends on the current **state** and **action**
        - and it is now called **context** ($c_t$)
    - in RL:
        - $(s_{t+1}, r_{t}) = f(a_t, s_t)$
    - in CMAB:
        - $r_{t} = f(a_t, c_t)$
        - $c_{t+1}$ is independent

## What are CMAB?
- generalisation of classification problem
- we don't have any training dataset, but can define **environment**
    - we get context - $c_t$ (observation)
    - we are asked to select one out of $N$ actions - $a_t$ (prediction)
    - we receive reward - $r_t$
- we collect data by interacting with **environment**
    - and train our **agent** on this experience

## Agents
- consist of:
    - **oracle** - regression model that predicts the `reward` of each `action` based on `context`
        - $\hat r_{a, t} = o_{a}(c_t)$
        - this can be generally any supervised regression model (linear regression, neural net)
    - **policy** - decide on the `action` based on predictions from oracle
        - $\hat a_t = p(\hat r_{a_1,t}, \hat r_{a_2,t}, ..., \hat r_{a_N,t})$
        - we have a few general types that we will review now

## $\epsilon$-Greedy (eps)
- how does it work:
    - select random action with probability $\epsilon$
    - otherwise, select the action with highest predicted reward
- needs just oracle's point estimates!
- very simple, but suboptimal
    - will continue exploring forever
    - there are extentions that try to alleviate - e.g. time-decay, adaptive greedy

## Boltzman Exploration
- how does it work:
    - calculate probability of each action based on it's predicted reward        
        - $P(a,t) = \frac{exp(\frac{\hat r_{a, t}}{\tau})}{\sum_{i=1}^N exp(\frac{\hat r_{a, t}}{\tau})}$
    - sample with these probabilities as weights
    - parameter $\tau$ (temperature) - used to control the spread of softmax
- needs just oracle's point estimates!
- more flexible than **eps**

## Upper Confidence Bound (UCB)
- how does it work:
    - combine estimated mean (point estimate) and variance of each action into UCB value
    - $UCB_{a, t} = \hat r_{a, t} + \alpha \frac{Var(\hat r_{a, t})}{n_a}$
    - $\hat a_t = \underset{a_t \in \{a_1, a_2, ..., a_N\} }{\operatorname{argmax}} UCB_{a,t}$
    - parameter $\alpha$ - used to control the amount of exploration
- requires oracle to provide **variance** of the estimate as well!
    - this is easy for linear regression
    - not so much for non-parametric methods

## Thompson Sampling (TS)
- how does it work:
    - define reward distribution of each action using estimated mean (point estimate) and variance
        - $D_{a, t} = \mathcal{N}(\hat r_{a, t}, \alpha \frac{Var(\hat r_{a, t})}{n_a})$
    - draw a sample ($\hat r^{TS}_{a_1,t}$) from each distribution and select argmax
        - $\hat a_t = \underset{a_t \in \{a_1, a_2, ..., a_N\} }{\operatorname{argmax} \{\hat r^{TS}_{a_1,t}, \hat r^{TS}_{a_2,t}, ... \hat r^{TS}_{a_N,t}\}} $
    - parameter $\alpha$ - used to control the amount of exploration
- in practise we just use normal, but you could potentially select arbitrary distribution
- requires oracle to provide **variance** of the estimate as well!
    - this is easy for linear regression
    - not so much for non-parametric methods

## CMAB summarised
- for typical classification problems (independent observations)
- define environment (with reward function) instead of collecting "true" labels
- train regression models to map the context to reward (oracle)
- utilise decission function to select action based on the expected rewards (policy)
    - responsible for balancing exploration and exploitation
- iteratively collect data and train the model

# TF-Agents

## More terms
- **time_step** - single output from **environment** for **agent**
    - `state (S)` - `reward (R)` - `discount` - `step type`
    - e.g. state of chessboard + reward for previous action
- **policy_step** - single output from **agent**
    - `action (A)` - `policy info`
- **epoch** - a group of timesteps
    - e.g. one game of chess
- **trajectory** - collected values of `S`, `A`, `R` from observed timesteps
    - e.g. `namedtuple` of `numpy.arrays`
    - collected during the **environment** - **state** iterations and used for training

<img src="www/tf_agents.png">

###### credits: [Inside TF-Agents](https://youtu.be/U7g7-Jzj9qo?t=1450)

## TensorFlow cheatsheet
- [tf.cast](https://www.tensorflow.org/api_docs/python/tf/cast) - change data type (TensorFlow is very strict about `dtypes`)
- [tf.math.bincount](https://www.tensorflow.org/api_docs/python/tf/math/bincount) - count occurence of each number from `0` to `maxlength`
    - `tf.math.bincount([0, 0, 2, 3, 3, 5]) -> [2, 0, 1, 2, 0, 1]`
- [tf.math.reduce_{max,min,mean,...}](https://www.tensorflow.org/api_docs/python/tf/math/reduce_sum) - instead of classical `sum`, `max`, ...
- [tf.boolean_mask](https://www.tensorflow.org/api_docs/python/tf/boolean_mask) - filter data Tensor using boolean Tensor 
- [tf.where](https://www.tensorflow.org/api_docs/python/tf/where) - like numpy equivalent
- [tf.one_hot](https://www.tensorflow.org/api_docs/python/tf/one_hot) - one-hot encoding of Tensor